In [ ]:
# hide
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# export
from typing import Collection, Counter

import fastparquet
import pandas as pd
import numpy as np
import torch

from fastai2.basics import (
    Chunks,
    ReindexCollection,
    round_multiple,
    delegates,
    tuplify,
    Tuple,
    IndexSplitter,
    DataSource,
    DataBunch,
    attrgetter,
    range_of,
    Cuda,
)
from fastai2.text.data import (
    LMTensorText,
    tokenize_df,
    BaseTokenizer,
    make_vocab,
    LMDataLoader,
    Numericalize,
)

<IPython.core.display.Javascript object>

In [ ]:
# default_exp data.preprocessing

<IPython.core.display.Javascript object>

# Data pre-processing

> From raw parquet to an ML-suitable, clean dataset

Once the ETL runs, we'll have a bunch of parquet files representing MIDI songs. However, there is still a little work to do to turn those into a format that is suitable for training a model.

Let's go through the steps we'll need to do.

## Reading raw parquet files into raw dataframes

First order of business is turning the output of the data ETL into a dataframe ready for processing.

In [ ]:
# export


def read_parquet(path: str) -> pd.DataFrame:
    """
    Reads a multi-file parquet at `path`, returning a dataframe of three columns.
    """
    df = fastparquet.ParquetFile(path, verify=True).to_pandas()
    return df

<IPython.core.display.Javascript object>

Next we need to tokenize the contents, and get ahold of the counters.

In [ ]:
# export


def preprocess(df: pd.DataFrame) -> (pd.DataFrame, Counter[str], Counter[str]):
    """
    Tokenizes pitches and durations and returns a dataframe
    """
    df.pitches = df.pitches.apply(lambda x: " ".join(map(str, x)))
    df.durations = df.durations.apply(lambda x: " ".join(map(str, x)))
    df = df[df["pitches"].apply(lambda x: len(x) != 0)].reset_index()
    df_tok, pitch_count = tokenize_df(df, "pitches", rules=[], tok_func=BaseTokenizer)
    df_tok["pitches"] = df_tok["text"]
    df_tok, duration_count = tokenize_df(
        df_tok, "durations", rules=[], tok_func=BaseTokenizer
    )
    df_tok["durations"] = df_tok["text"]
    df_tok = df_tok.drop("text", axis=1).drop("index", axis=1)

    return df_tok, pitch_count, duration_count

<IPython.core.display.Javascript object>

In [ ]:
# test
from testing import test_eq, path

from neuralmusic.midi import parse_midi_file

df, _ = parse_midi_file(path("data/ff4-airship.mid"))
df_tok, pitch_count, duration_count = preprocess(df)

test_eq(["7.11.2", "7", "7"], list(df_tok["pitches"][0][0:3]))
test_eq(["1.75", "0.5", "0.5"], list(df_tok["durations"][0][0:3]))
test_eq([110, 110, 110], list(df_tok["velocities"][0][0:3]))

test_eq(43, len(pitch_count))
test_eq(8, len(duration_count))

<IPython.core.display.Javascript object>

## Transforms

When constructing our data source, we'll build some transforms to first get tuples of values at a time (pitches and durations), and numericalize them in parallel as well.

In [ ]:
# export
Vocab = Collection[str]


def to_dual(fields):
    """
    Returns a transform that will extract `fields` from a Series in the form of fastai Tuples.
    """
    getters = [attrgetter(field) for field in fields]

    def _inner(series: pd.Series):
        return Tuple(tuplify([getter(series) for getter in getters]))

    return _inner


def dual_numericalize(vocabs: Collection[Vocab]):
    """
    Returns a transform that will numericalize each side of the tuple constructing a separate
    vocabulary for each side.
    """
    processors = [Numericalize(vocab) for vocab in vocabs]

    def _inner(values):
        return [proc(val) for (proc, val) in zip(processors, values)]

    return _inner

<IPython.core.display.Javascript object>

We also need to make a splitter that will separate our rows according to a certain ratio, by default 0.2.

In [ ]:
# export


def make_splitter(df: pd.DataFrame, split: float = 0.2) -> IndexSplitter:
    """
    Returns a splitter that acts on indices on a dataframe. By default it reserves 20% of the
    data for validation.
    """
    rows = len(df)
    indices = np.array(range(rows))
    np.random.shuffle(indices)
    _, valid_idx = (indices[int(0.2 * rows) :], indices[: int(0.2 * rows)])
    return IndexSplitter(valid_idx)

<IPython.core.display.Javascript object>

## DataLoader

We need a slightly custom DataLoader that instead of loading single sequences of tokens like in traditional language models, loads tuples of sequences (in our case, a sequence of pitches and a sequence of durations), all at the same time.

In [ ]:
# export
@delegates()
class DualLMDataLoader(LMDataLoader):
    """
    A Language Model data loader that loads tuples of 2 sequences instead of single sequences.
    It's used to load pitches and durations at the same time.
    """

    def __init__(
        self, dataset, lens=None, cache=2, bs=64, seq_len=72, num_workers=0, **kwargs
    ):
        super().__init__(dataset=dataset, bs=bs, num_workers=num_workers, **kwargs)
        self.items = ReindexCollection(
            [(o[0] if isinstance(o, tuple) else o) for o in dataset], cache=cache
        )
        self.seq_len = seq_len
        if lens is None:
            lens = [len(o[0]) for o in self.items]
        self.lens = ReindexCollection(lens, idxs=self.items.idxs)
        # The "-1" is to allow for final label
        self.m = round_multiple(sum(lens) - 1, bs * seq_len, round_down=True)
        self.n = self.m // (seq_len)
        self.spb = self.n // bs
        self.make_chunks()

    def make_chunks(self):
        self.a_chunks = Chunks(list(map(lambda x: x[0], self.items)), self.lens)
        self.b_chunks = Chunks(list(map(lambda x: x[1], self.items)), self.lens)

    def create_item(self, seq):
        if seq >= self.n:
            raise IndexError
        st = ((seq % self.bs) * self.spb + (seq // self.bs)) * self.seq_len
        txt_a = self.a_chunks[st : st + self.seq_len + 1]
        txt_b = self.b_chunks[st : st + self.seq_len + 1]
        x1 = LMTensorText(txt_a[:-1]).unsqueeze(-2)
        x2 = LMTensorText(txt_b[:-1]).unsqueeze(-2)
        y1 = txt_a[1:].unsqueeze(-2)
        y2 = txt_b[1:].unsqueeze(-2)
        return torch.cat([x1, x2]), torch.cat([y1, y2])

<IPython.core.display.Javascript object>

Finally we bring everything together by creating a DataSource.

In [ ]:
# export


def data_source(
    df: pd.DataFrame,
    pitch_vocab: Vocab,
    duration_vocab: Vocab,
    split: float = 0.2,
    dl_type=DualLMDataLoader,
) -> DataSource:
    """
    Creates a DataSource ready to become a databunch.
    """
    splitter = make_splitter(df, split=split)
    return DataSource(
        df,
        [
            [
                to_dual(["pitches", "durations"]),
                dual_numericalize([pitch_vocab, duration_vocab]),
                Cuda(),
            ]
        ],
        splits=splitter(range_of((df))),
        dl_type=dl_type,
    )

<IPython.core.display.Javascript object>

# Bringing everything together

Now we can obtain a DataBunch ready for training from a bunch of parquet files just like that:

In [ ]:
# export


def process(
    path: str, batch_size: int, seq_len: int, validation_split: float = 0.2
) -> (DataBunch, Vocab, Vocab):
    """
    Turn raw parquet files into a DataBunch ready for training.
    """
    raw_df = read_parquet(path)
    df, pitch_count, duration_count = preprocess(raw_df)
    pitch_vocab = make_vocab(pitch_count, min_freq=1)
    duration_vocab = make_vocab(duration_count, min_freq=1)
    batch_size = batch_size if batch_size < len(df) else len(df)
    dsrc = data_source(df, pitch_vocab, duration_vocab, split=validation_split)
    return (
        dsrc.databunch(bs=batch_size, seq_len=seq_len, after_batch=Cuda()),
        pitch_vocab,
        duration_vocab,
    )

<IPython.core.display.Javascript object>